<a href="https://colab.research.google.com/github/hj-22/2025-1-DM_Proj/blob/jiwon/bird_variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile

# 공항별 eBird data 만들기

In [54]:
### bird data
## 종별 migratory 여부 (population size는 2025년 현재 기준)
is_migratory = pd.read_csv("/content/drive/MyDrive/DM_project/birdlife_data.csv")
print(is_migratory.columns)
print(is_migratory.isnull().sum().to_string())

## eBird data per airport
cols =['TAXONOMIC ORDER',
        'CATEGORY', 'SCIENTIFIC NAME', 'COMMON NAME', 'OBSERVATION COUNT',
        'OBSERVATION DATE', 'OBSERVER ID', 'DURATION MINUTES', 'EFFORT DISTANCE KM']
ksmf = pd.read_csv('/content/drive/MyDrive/DM_project/filtered_KSMF.txt', sep='\t', usecols=cols)
klax = pd.read_csv('/content/drive/MyDrive/DM_project/filtered_KLAX.txt', sep='\t', usecols=cols)
ksfo = pd.read_csv('/content/drive/MyDrive/DM_project/filtered_KSFO.txt', sep='\t', usecols=cols)
koak = pd.read_csv('/content/drive/MyDrive/DM_project/filtered_KOAK.txt', sep='\t', usecols=cols)
ksjc = pd.read_csv('/content/drive/MyDrive/DM_project/filtered_KSJC.txt', sep='\t', usecols=cols)
ksan = pd.read_csv('/content/drive/MyDrive/DM_project/filtered_KSAN.txt', sep='\t', usecols=cols)
ksna = pd.read_csv('/content/drive/MyDrive/DM_project/filtered_KSNA.txt', sep='\t', usecols=cols)

Index(['SIS ID', 'Sequence', 'Family', 'Scientific name', 'Common name',
       'RL Category', 'PE', 'PEW', 'Seabird', 'Waterbird', 'Landbird',
       'Migratory status', 'Ecosystem - Terrestrial', 'Ecosystem - Freshwater',
       'Ecosystem - Marine', 'RL AOO', 'Criteria met at highest level',
       'RL EOO (smaller of breeding and non-breeding EOO)',
       'Population size (mature individuals)', 'Population size derivation',
       'Current population trend', 'Current population trend derivation'],
      dtype='object')
SIS ID                                                  0
Sequence                                                0
Family                                                  0
Scientific name                                         0
Common name                                             0
RL Category                                             0
PE                                                      0
PEW                                                     0
Seabir

In [56]:
ksmf = ksmf[(ksmf["DURATION MINUTES"]<=180)&(ksmf["EFFORT DISTANCE KM"]<=5)]
ksmf = ksmf[["OBSERVATION DATE","COMMON NAME", "SCIENTIFIC NAME", "OBSERVATION COUNT", "OBSERVER ID"]]

klax = klax[(klax["DURATION MINUTES"]<=180)&(klax["EFFORT DISTANCE KM"]<=5)]
klax = klax[["OBSERVATION DATE", "COMMON NAME", "SCIENTIFIC NAME", "OBSERVATION COUNT", "OBSERVER ID"]]

ksfo = ksfo[(ksfo["DURATION MINUTES"]<=180)&(ksfo["EFFORT DISTANCE KM"]<=5)]
ksfo = ksfo[["OBSERVATION DATE", "COMMON NAME", "SCIENTIFIC NAME", "OBSERVATION COUNT", "OBSERVER ID"]]

koak = koak[(koak["DURATION MINUTES"]<=180)&(koak["EFFORT DISTANCE KM"]<=5)]
koak = koak[["OBSERVATION DATE", "COMMON NAME", "SCIENTIFIC NAME", "OBSERVATION COUNT", "OBSERVER ID"]]

ksjc = ksjc[(ksjc["DURATION MINUTES"]<=180)&(ksjc["EFFORT DISTANCE KM"]<=5)]
ksjc = ksjc[["OBSERVATION DATE", "COMMON NAME", "SCIENTIFIC NAME", "OBSERVATION COUNT", "OBSERVER ID"]]

ksan = ksan[(ksan["DURATION MINUTES"]<=180)&(ksan["EFFORT DISTANCE KM"]<=5)]
ksan = ksan[["OBSERVATION DATE", "COMMON NAME", "SCIENTIFIC NAME", "OBSERVATION COUNT", "OBSERVER ID"]]

ksna = ksna[(ksna["DURATION MINUTES"]<=180)&(ksna["EFFORT DISTANCE KM"]<=5)]
ksna = ksna[["OBSERVATION DATE", "COMMON NAME", "SCIENTIFIC NAME", "OBSERVATION COUNT", "OBSERVER ID"]]

In [57]:
fivedays_dict = dict(zip(list(range(1,366)), [i//5+1 if i%5!=0 else i//5 for i in range(1,366)]))
for df in [ksmf, klax, ksfo, koak, ksjc, ksan, ksna]:
  df["YEAR"] = pd.to_datetime(df["OBSERVATION DATE"]).dt.year
  df["MONTH"] = pd.to_datetime(df["OBSERVATION DATE"]).dt.month
  df["DAY_OF_YEAR"] = pd.to_datetime(df["OBSERVATION DATE"]).dt.dayofyear
  for year in [2004, 2008, 2012, 2016, 2020, 2024]:
    for month in range(3, 13):
      df.loc[(df["YEAR"]==year)&(df["MONTH"]==month), "DAY_OF_YEAR"] -= 1
  df["FIVEDAY"] = df["DAY_OF_YEAR"].apply(lambda x: fivedays_dict[x])

ksfo.head()


,OBSERVATION DATE,COMMON NAME,SCIENTIFIC NAME,OBSERVATION COUNT,OBSERVER ID,YEAR,MONTH,DAY_OF_YEAR,FIVEDAY
892,2000-03-28,Allen's Hummingbird,Selasphorus sasin,1,obsr102121,2000,3,88,18
894,2000-03-28,American Robin,Turdus migratorius,X,obsr102121,2000,3,88,18
898,2000-03-28,American Wigeon,Mareca americana,X,obsr102121,2000,3,88,18
907,2000-03-28,Brewer's Blackbird,Euphagus cyanocephalus,X,obsr102121,2000,3,88,18
918,2000-03-28,Chestnut-backed Chickadee,Poecile rufescens,X,obsr102121,2000,3,88,18


In [58]:
is_migratory["Migratory"] = is_migratory["Migratory status"].apply(lambda x: 1 if x=="Full migrant" else 0)
is_migratory["Residential"] = is_migratory["Migratory status"].apply(lambda x: 1 if x=="Not a migrant" else 0)

is_waterbird = dict(zip(is_migratory["Scientific name"],is_migratory["Waterbird"].astype(int)))
is_landbird = dict(zip(is_migratory["Scientific name"],is_migratory["Landbird"].astype(int)))
is_migratory_bird = dict(zip(is_migratory["Scientific name"],is_migratory["Migratory"]))
is_residential_bird = dict(zip(is_migratory["Scientific name"],is_migratory["Residential"]))

In [81]:
### migratory count, residential count, waterbird count, landbird count
## 2006~2024
airport_df = {"KSMF":ksmf, "KSFO":ksfo, "KLAX":klax, "KOAK":koak, "KSJC":ksjc, "KSAN": ksan, "KSNA":ksna}

num_migratory_birds = dict()
num_residential_birds = dict()
num_landbirds = dict()
num_waterbirds = dict()
num_observers = dict()
migratory_abundance = dict()
residential_abundance = dict()

# create migratory
for airport in airport_df.keys():
  df = airport_df[airport]

  df["OBSERVATION COUNT"] = df["OBSERVATION COUNT"].apply(lambda x: x if x!="X" else "0")
  df["OBSERVATION COUNT"] = df["OBSERVATION COUNT"].astype(int)
  df["IS_MIGRATORY"] = df["SCIENTIFIC NAME"].apply(lambda x: is_migratory_bird.get(x, 0))
  df["IS_RESIDENTIAL"] = df["SCIENTIFIC NAME"].apply(lambda x: is_residential_bird.get(x, 0))
  df["IS_LANDBIRD"] = df["SCIENTIFIC NAME"].apply(lambda x: is_landbird.get(x, 0))
  df["IS_WATERBIRD"] = df["SCIENTIFIC NAME"].apply(lambda x: is_waterbird.get(x, 0))

  # COMMON NAME=="Snow/Ross's Goose"인 행 처리
  df.loc[df["COMMON NAME"]=="Snow/Ross's Goose", "IS_MIGRATORY"] = 1
  df.loc[df["COMMON NAME"]=="Snow/Ross's Goose", "IS_RESIDENTIAL"] = 0
  df.loc[df["COMMON NAME"]=="Snow/Ross's Goose", "IS_LANDBIRD"] = 0
  df.loc[df["COMMON NAME"]=="Snow/Ross's Goose", "IS_WATERBIRD"] = 1

  df["MIGRATORY_COUNT"] = df["OBSERVATION COUNT"] * df["IS_MIGRATORY"]
  df["RESIDENTIAL_COUNT"] = df["OBSERVATION COUNT"] * df["IS_RESIDENTIAL"]
  df["LANDBIRD_COUNT"] = df["OBSERVATION COUNT"] * df["IS_LANDBIRD"]
  df["WATERBIRD_COUNT"] = df["OBSERVATION COUNT"] * df["IS_WATERBIRD"]

  df_grouped = df[df["YEAR"]>=2006].groupby(by=["YEAR", "FIVEDAY"])
  num_migratory_birds[airport] = df_grouped["MIGRATORY_COUNT"].sum().to_dict()
  num_residential_birds[airport] = df_grouped["RESIDENTIAL_COUNT"].sum().to_dict()
  num_waterbirds[airport] = df_grouped["WATERBIRD_COUNT"].sum().to_dict()
  num_landbirds[airport] = df_grouped["LANDBIRD_COUNT"].sum().to_dict()
  num_observers[airport] = df_grouped["OBSERVER ID"].nunique().to_dict()
  migratory_abundance[airport] = df[(df["YEAR"]>=2006) & (df["IS_MIGRATORY"]==1)].groupby(by=["YEAR", "FIVEDAY"])["SCIENTIFIC NAME"].nunique().to_dict()
  residential_abundance[airport] =df[(df["YEAR"]>=2006) & (df["IS_RESIDENTIAL"]==1)].groupby(by=["YEAR", "FIVEDAY"])["SCIENTIFIC NAME"].nunique().to_dict()


In [82]:
ksmf.head()

,OBSERVATION DATE,COMMON NAME,SCIENTIFIC NAME,OBSERVATION COUNT,OBSERVER ID,YEAR,MONTH,DAY_OF_YEAR,FIVEDAY,IS_MIGRATORY,IS_RESIDENTIAL,IS_LANDBIRD,IS_WATERBIRD,MIGRATORY_COUNT,RESIDENTIAL_COUNT,LANDBIRD_COUNT,WATERBIRD_COUNT
233,2000-04-28,American Crow,Corvus brachyrhynchos,0,obsr129389,2000,4,119,24,1,0,1,0,0,0,0,0
237,2000-04-28,American Goldfinch,Spinus tristis,0,obsr129389,2000,4,119,24,1,0,1,0,0,0,0,0
239,2000-04-28,American Kestrel,Falco sparverius,0,obsr129389,2000,4,119,24,1,0,1,0,0,0,0,0
243,2000-04-28,American Robin,Turdus migratorius,0,obsr129389,2000,4,119,24,1,0,1,0,0,0,0,0
246,2000-04-28,Anna's Hummingbird,Calypte anna,0,obsr129389,2000,4,119,24,1,0,1,0,0,0,0,0


In [83]:
ksmf[ksmf["COMMON NAME"]=="Snow/Ross's Goose"]

,OBSERVATION DATE,COMMON NAME,SCIENTIFIC NAME,OBSERVATION COUNT,OBSERVER ID,YEAR,MONTH,DAY_OF_YEAR,FIVEDAY,IS_MIGRATORY,IS_RESIDENTIAL,IS_LANDBIRD,IS_WATERBIRD,MIGRATORY_COUNT,RESIDENTIAL_COUNT,LANDBIRD_COUNT,WATERBIRD_COUNT
36955,2009-04-23,Snow/Ross's Goose,Anser caerulescens/rossii,20,obsr183113,2009,4,113,23,1,0,0,1,20,0,0,20
49938,2010-01-09,Snow/Ross's Goose,Anser caerulescens/rossii,200,obsr123137,2010,1,9,2,1,0,0,1,200,0,0,200
78416,2011-01-19,Snow/Ross's Goose,Anser caerulescens/rossii,50,obsr34738,2011,1,19,4,1,0,0,1,50,0,0,50
169797,2012-12-26,Snow/Ross's Goose,Anser caerulescens/rossii,2500,obsr152314,2012,12,360,72,1,0,0,1,2500,0,0,2500
169798,2012-12-26,Snow/Ross's Goose,Anser caerulescens/rossii,1500,obsr152314,2012,12,360,72,1,0,0,1,1500,0,0,1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1548741,2023-03-10,Snow/Ross's Goose,Anser caerulescens/rossii,0,obsr3364367,2023,3,69,14,1,0,0,1,0,0,0,0
1672436,2023-12-22,Snow/Ross's Goose,Anser caerulescens/rossii,300,obsr28733,2023,12,356,72,1,0,0,1,300,0,0,300
1854083,2024-12-01,Snow/Ross's Goose,Anser caerulescens/rossii,1,obsr3563199,2024,12,335,67,1,0,0,1,1,0,0,1
1854084,2024-12-22,Snow/Ross's Goose,Anser caerulescens/rossii,14,obsr831853,2024,12,356,72,1,0,0,1,14,0,0,14


In [61]:
master_df = pd.read_csv("/content/drive/MyDrive/DM_project/master.csv")
master_df.head()

,AIRPORT_ID,YEAR,FIVEDAY,AIRPORT_LATITUDE,AIRPORT_LONGITUDE,RESPONSE,WND_mean,CIG_mean,VIS_mean,TMP_mean,DEW_mean,SLP_mean,RAIN_mean,WND_sd,CIG_sd,VIS_sd,TMP_sd,DEW_sd,SLP_sd,RAIN_sd
0,KSMF,2009,1,38.69542,-121.59077,0.000000,27.756579,7992.867550,8839.335526,46.900662,22.655629,10199.483333,0.198413,23.000719,10345.942924,6126.517876,32.723642,34.557594,37.021453,0.848715
1,KSMF,2009,2,38.69542,-121.59077,0.001065,16.919753,9841.496894,8086.296296,55.372671,37.645963,10262.825000,0.064000,13.735287,10833.811412,5497.274976,33.242634,22.947606,34.437292,0.715542
2,KSMF,2009,3,38.69542,-121.59077,0.004162,8.697368,18363.801325,8312.572368,60.377483,31.251656,10268.450000,0.000000,19.080186,8190.414659,6053.373697,64.068062,29.932640,39.653298,0.000000
3,KSMF,2009,4,38.69542,-121.59077,0.003304,10.370629,20924.545455,11554.972028,71.188811,31.951049,10240.266667,0.000000,12.821460,4757.034928,5499.695579,60.275185,22.354799,18.636610,0.000000
4,KSMF,2009,5,38.69542,-121.59077,0.001092,22.803797,5344.702532,9327.367089,103.911392,85.632911,10175.591667,4.296296,19.148917,8640.255394,5832.868399,23.949091,27.428275,10.497295,17.206418


In [84]:
### create bird variables
master_df["MIGRATORY_COUNT"] = master_df.index.map(lambda i: num_migratory_birds[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item(), master_df.loc[i, "FIVEDAY"].item()), 0) / num_observers[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item(), master_df.loc[i, "FIVEDAY"].item()), 1e-5))
master_df["RESIDENT_COUNT"] = master_df.index.map(lambda i: num_residential_birds[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item(), master_df.loc[i, "FIVEDAY"].item()), 0) / num_observers[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item(), master_df.loc[i, "FIVEDAY"].item()), 1e-5))
master_df["WATERBIRD_COUNT"] = master_df.index.map(lambda i: num_landbirds[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item(), master_df.loc[i, "FIVEDAY"].item()), 0) / num_observers[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item(), master_df.loc[i, "FIVEDAY"].item()), 1e-5))
master_df["LANDBIRD_COUNT"] = master_df.index.map(lambda i: num_waterbirds[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item(), master_df.loc[i, "FIVEDAY"].item()), 0) / num_observers[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item(), master_df.loc[i, "FIVEDAY"].item()), 1e-5))

# lag1
master_df["MIGRATORY_COUNT_1"] = master_df.index.map(lambda i: num_migratory_birds[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-1, master_df.loc[i, "FIVEDAY"].item()), 0) / num_observers[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-1, master_df.loc[i, "FIVEDAY"].item()), 1e-5))
master_df["RESIDENT_COUNT_1"] = master_df.index.map(lambda i: num_residential_birds[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-1, master_df.loc[i, "FIVEDAY"].item()), 0) / num_observers[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-1, master_df.loc[i, "FIVEDAY"].item()), 1e-5))
master_df["WATERBIRD_COUNT_1"] = master_df.index.map(lambda i: num_landbirds[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-1, master_df.loc[i, "FIVEDAY"].item()), 0) / num_observers[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-1, master_df.loc[i, "FIVEDAY"].item()), 1e-5))
master_df["LANDBIRD_COUNT_1"] = master_df.index.map(lambda i: num_waterbirds[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-1, master_df.loc[i, "FIVEDAY"].item()), 0) / num_observers[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-1, master_df.loc[i, "FIVEDAY"].item()), 1e-5))

# lag2
master_df["MIGRATORY_COUNT_2"] = master_df.index.map(lambda i: num_migratory_birds[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-2, master_df.loc[i, "FIVEDAY"].item()), 0) / num_observers[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-2, master_df.loc[i, "FIVEDAY"].item()), 1e-5))
master_df["RESIDENT_COUNT_2"] = master_df.index.map(lambda i: num_residential_birds[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-2, master_df.loc[i, "FIVEDAY"].item()), 0) / num_observers[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-2, master_df.loc[i, "FIVEDAY"].item()), 1e-5))
master_df["WATERBIRD_COUNT_2"] = master_df.index.map(lambda i: num_landbirds[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-2, master_df.loc[i, "FIVEDAY"].item()), 0) / num_observers[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-2, master_df.loc[i, "FIVEDAY"].item()), 1e-5))
master_df["LANDBIRD_COUNT_2"] = master_df.index.map(lambda i: num_waterbirds[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-2, master_df.loc[i, "FIVEDAY"].item()), 0) / num_observers[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-2, master_df.loc[i, "FIVEDAY"].item()), 1e-5))

# lag3
master_df["MIGRATORY_COUNT_3"] = master_df.index.map(lambda i: num_migratory_birds[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-3, master_df.loc[i, "FIVEDAY"].item()), 0) / num_observers[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-3, master_df.loc[i, "FIVEDAY"].item()), 1e-5))
master_df["RESIDENT_COUNT_3"] = master_df.index.map(lambda i: num_residential_birds[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-3, master_df.loc[i, "FIVEDAY"].item()), 0) / num_observers[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-3, master_df.loc[i, "FIVEDAY"].item()), 1e-5))
master_df["WATERBIRD_COUNT_3"] = master_df.index.map(lambda i: num_landbirds[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-3, master_df.loc[i, "FIVEDAY"].item()), 0) / num_observers[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-3, master_df.loc[i, "FIVEDAY"].item()), 1e-5))
master_df["LANDBIRD_COUNT_3"] = master_df.index.map(lambda i: num_waterbirds[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-3, master_df.loc[i, "FIVEDAY"].item()), 0) / num_observers[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item()-3, master_df.loc[i, "FIVEDAY"].item()), 1e-5))

# abundance
master_df["MIGRATORY_ABUN"] = master_df.index.map(lambda i: migratory_abundance[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item(), master_df.loc[i, "FIVEDAY"].item()), 0))
master_df["RESIDENTIAL_ABUN"] = master_df.index.map(lambda i: residential_abundance[master_df.loc[i,"AIRPORT_ID"]].get((master_df.loc[i, "YEAR"].item(), master_df.loc[i, "FIVEDAY"].item()), 0))


In [85]:
master_df.columns

Index(['AIRPORT_ID', 'YEAR', 'FIVEDAY', 'AIRPORT_LATITUDE',
       'AIRPORT_LONGITUDE', 'RESPONSE', 'WND_mean', 'CIG_mean', 'VIS_mean',
       'TMP_mean', 'DEW_mean', 'SLP_mean', 'RAIN_mean', 'WND_sd', 'CIG_sd',
       'VIS_sd', 'TMP_sd', 'DEW_sd', 'SLP_sd', 'RAIN_sd', 'MIGRATORY_COUNT',
       'RESIDENT_COUNT', 'WATERBIRD_COUNT', 'LANDBIRD_COUNT',
       'MIGRATORY_COUNT_1', 'RESIDENT_COUNT_1', 'WATERBIRD_COUNT_1',
       'LANDBIRD_COUNT_1', 'MIGRATORY_COUNT_2', 'RESIDENT_COUNT_2',
       'WATERBIRD_COUNT_2', 'LANDBIRD_COUNT_2', 'MIGRATORY_COUNT_3',
       'RESIDENT_COUNT_3', 'WATERBIRD_COUNT_3', 'LANDBIRD_COUNT_3',
       'MIGRATORY_ABUN', 'RESIDENTIAL_ABUN'],
      dtype='object')

In [79]:
master_df.to_csv("master_with_bird.csv")